### LOAD MODEL

In [2]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM
from llm_config import SUPPORTED_LLM_MODELS

model_configuration = SUPPORTED_LLM_MODELS["English"]["phi-3-mini-instruct"]

model_dir = "./phi-3-mini-instruct/INT4_compressed_weights"
print(f"Loading model from {model_dir}")

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

# if "GPU" in device.value and "qwen2-7b-instruct" in model_id.value:
#     ov_config["GPU_ENABLE_SDPA_OPTIMIZATION"] = "NO"

# On a GPU device a model is executed in FP16 precision. For red-pajama-3b-chat model there known accuracy
# issues caused by this, which we avoid by setting precision hint to "f32".
# if model_id.value == "red-pajama-3b-chat" and "GPU" in core.available_devices and device.value in ["GPU", "AUTO"]:
#     ov_config["INFERENCE_PRECISION_HINT"] = "f32"

model_name = model_configuration["model_id"]
tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device="CPU",
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

Loading model from ./phi-3-mini-instruct/INT4_compressed_weights


Compiling the model to CPU ...


In [3]:
# Example text to be processed
input_text = "Hello, how are you doing today?"

# Tokenize the text
inputs = tok(input_text, return_tensors="pt")

# Perform inference
outputs = ov_model.generate(**inputs)

# Decode the generated tokens to text
decoded_output = tok.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", decoded_output)

c:\Users\intelaipc\miniforge3\envs\openvino2\Lib\site-packages\transformers\generation\utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Text: Hello, how are you doing today? I'm doing well, thank you for asking. I


### MULTI SHOT LEARNING: STORY TELLING

In [ ]:
def generate_response_x(prompt, examples):
    # Combine examples with the prompt
    formatted_prompt = "".join([f"{ex['input']}: {ex['output']}\n" for ex in examples]) + prompt
    responses = pipeline(formatted_prompt, max_new_tokens=100, num_return_sequences=1)
    return responses[0]["generated_text"]

# Example usage. examples taken from https://huggingface.co/datasets/ajibawa-2023/Children-Stories-Collection
examples = [
    {"input": "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science.", "output": "Once upon a time in the land of Policymia, there lived two leaders named Majora and Minoro. Their job was to make sure all the citizens had beautiful parks, clean water, and top-notch schools. But there were so many things to fix! How would they ever decide where to start? Majora, being the wise leader she was, knew just what to do. She invited her fellow policymakers for a big meeting at the Roundtable of Representatives. There, they discussed the most important problems Policymia faced. This was called identifying 'key policy areas.' It meant figuring out which topics needed attention first. Next came assessing support – finding out if everyone agreed on the solutions. Some people thought building more playgrounds was the way to go, while others wanted better libraries. To understand everyone's thoughts, Majora used something called 'polling.' Just like taking a vote, polling helped her see what ideas were popular among her friends (the majority) and also those across the aisle (people who didn't belong to her political group). While talking to her friends and colleagues, Majora discovered that almost everyone loved science! And why not? Science could help create amazing inventions, protect nature, and even cure sicknesses. So, the policymakers decided to build a super cool SCIENCE CENTER right at the heart of Policymia! But then, an unexpected problem popped up! A grumpy neighboring kingdom threatened to block Policymia's plans because they feared losing visitors to the new center. Oh no! However, instead of giving up, Majora saw this challenge as an opportunity. If they could work together with the grumpy neighbors, maybe both lands could benefit. That way, everybody wins, showing the true spirit of teamwork and collaboration in the world of policies!"},
    {"input": "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science.", "output": "In the heart of Harmonyville, nestled between two neighboring countries, lay a magnificent bridge. Not only did it connect the land masses, but it also joined hearts and minds of folks living on either side. Let's call our beloved structure 'The Friendship Bridge'. One bright morning, Sarah from Sunnyhill and Maria from Starshine decided to explore this marvelous architectural feat. As they skipped along its vibrant pathway adorned with multicolored flags fluttering cheerily, Sarah asked, 'Maria, do you know how bridges stand strong despite all the cars going over them?' With a curious smile, Maria replied, 'Well, I think it has something to do with physics! Bridges use different types of forces like tension, compression, and torsion to balance themselves out.' Sarah gasped, 'Wow! That sounds complicated. But then again,' she pointed towards the distant mountains, 'those tall peaks must face greater force by wind and snow. Yet, they stand firm too!' As the day wore on, they reached the middle of the bridge where an information kiosk stood proudly. There, they learned about the various parts of the bridge – abutments, piers, deck, trusses...it seemed almost alive! Each part played a crucial role just like cells in our bodies. Just when things couldn't get any better, dark clouds rolled in unexpectedly. Rain began pouring down drenching everyone including Sarah and Maria. Their excitement turned into worry as they realized they might not make it back home before dark. Suddenly, the once familiar route appeared ominous under the stormy sky. Despite being scared, they remembered what they had learned earlier. Just like structures need balance and harmony, so does nature. So instead of panicking, they took deep breaths calming themselves down. After all, even amidst turmoil, understanding science can bring clarity and peace. And sometimes, facing adversities hand-in-hand brings forth stronger friendships."},
]
prompt = "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science."
response = generate_response_x(prompt, examples)
print(response)

In [ ]:
prompt = "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science."

def generate_response_0(prompt, max_new_tokens=100):
    response = pipeline(prompt, max_new_tokens=max_new_tokens, num_return_sequences=1)
    return response[0]['generated_text']

print(generate_response_0(prompt))

### MULTISHOT LEARNING: QUESTION PROMPTING

In [ ]:
def generate_response_x(prompt, examples):
    # Combine examples with the prompt
    formatted_prompt = "".join([f"{ex['input']}: {ex['output']}\n" for ex in examples]) + prompt
    responses = pipeline(formatted_prompt, max_new_tokens=100, num_return_sequences=1)
    return responses[0]["generated_text"]

# Example usage. examples taken from https://huggingface.co/datasets/ajibawa-2023/Children-Stories-Collection
examples = [
    {"input": "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science.", "output": "Once upon a time in the land of Policymia, there lived two leaders named Majora and Minoro. Their job was to make sure all the citizens had beautiful parks, clean water, and top-notch schools. But there were so many things to fix! How would they ever decide where to start? Majora, being the wise leader she was, knew just what to do. She invited her fellow policymakers for a big meeting at the Roundtable of Representatives. There, they discussed the most important problems Policymia faced. This was called identifying 'key policy areas.' It meant figuring out which topics needed attention first. Next came assessing support – finding out if everyone agreed on the solutions. Some people thought building more playgrounds was the way to go, while others wanted better libraries. To understand everyone's thoughts, Majora used something called 'polling.' Just like taking a vote, polling helped her see what ideas were popular among her friends (the majority) and also those across the aisle (people who didn't belong to her political group). While talking to her friends and colleagues, Majora discovered that almost everyone loved science! And why not? Science could help create amazing inventions, protect nature, and even cure sicknesses. So, the policymakers decided to build a super cool SCIENCE CENTER right at the heart of Policymia! But then, an unexpected problem popped up! A grumpy neighboring kingdom threatened to block Policymia's plans because they feared losing visitors to the new center. Oh no! However, instead of giving up, Majora saw this challenge as an opportunity. If they could work together with the grumpy neighbors, maybe both lands could benefit. That way, everybody wins, showing the true spirit of teamwork and collaboration in the world of policies!"},
    {"input": "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science.", "output": "In the heart of Harmonyville, nestled between two neighboring countries, lay a magnificent bridge. Not only did it connect the land masses, but it also joined hearts and minds of folks living on either side. Let's call our beloved structure 'The Friendship Bridge'. One bright morning, Sarah from Sunnyhill and Maria from Starshine decided to explore this marvelous architectural feat. As they skipped along its vibrant pathway adorned with multicolored flags fluttering cheerily, Sarah asked, 'Maria, do you know how bridges stand strong despite all the cars going over them?' With a curious smile, Maria replied, 'Well, I think it has something to do with physics! Bridges use different types of forces like tension, compression, and torsion to balance themselves out.' Sarah gasped, 'Wow! That sounds complicated. But then again,' she pointed towards the distant mountains, 'those tall peaks must face greater force by wind and snow. Yet, they stand firm too!' As the day wore on, they reached the middle of the bridge where an information kiosk stood proudly. There, they learned about the various parts of the bridge – abutments, piers, deck, trusses...it seemed almost alive! Each part played a crucial role just like cells in our bodies. Just when things couldn't get any better, dark clouds rolled in unexpectedly. Rain began pouring down drenching everyone including Sarah and Maria. Their excitement turned into worry as they realized they might not make it back home before dark. Suddenly, the once familiar route appeared ominous under the stormy sky. Despite being scared, they remembered what they had learned earlier. Just like structures need balance and harmony, so does nature. So instead of panicking, they took deep breaths calming themselves down. After all, even amidst turmoil, understanding science can bring clarity and peace. And sometimes, facing adversities hand-in-hand brings forth stronger friendships."},
]
prompt = "Write an educational story (3-5 paragraphs) targeted at young children using simple words. The story should be inspired from this text snippet: {}. The story should have the following features: - Science integration: embed basic science concepts within the story, explaining them through the characters' adventures and discoveries. - Characters and dialogue: create memorable characters who engage in meaningful conversations, helping to explain and explore the science concepts. - Unexpected twist: conclude with a twist that doesn't resolve as hoped, but leaves a clear lesson about life and science."
response = generate_response_x(prompt, examples)
print(response)

### ZERO SHOT LEARNING: QUESTION PROMPTING